In [29]:
import numpy as np
import tensorflow as tf
import os
import cv2
import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from collections import deque
import datetime
import json
import csv
import shutil
from urllib.request import urlopen

In [30]:
import threading
import time


In [31]:
image_height, image_width = 64, 64
max_images_per_class = 8000
 
dataset_directory = "dataset"

classes_list = ["Fighting","Shooting","RoadAccidents","Robbery","Abuse","Arrest","Arson","Assault","Burglary","Explosion","Normal"]

 
model_output_size = len(classes_list)
print(model_output_size)

11


In [32]:
model = tf.keras.models.load_model('model_8000.h5')

In [33]:
def save_clip(frames):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc('V','P','8','0')  
    frame_rate=30
    output_video = cv2.VideoWriter("D:/hackathons/output.webm", fourcc, frame_rate, (width, height))

    for frame in frames:
        output_video.write(frame)
    output_video.release()

In [34]:
def get_location():
    try:
        # Create a geocoder instance
        url='http://ipinfo.io/json'
        response=urlopen(url)
        # Get the current GPS location
        data=json.load(response)
        coordinates=data['loc']
        city=data['city']
        return city,coordinates
    except:
        city,coordinates='NA','NA'
        return city,coordinates

In [35]:
def get_date_and_time():
    data = datetime.datetime.now()
    data=str(data)
    date,time=data.split(' ')
    date=date.replace('-','_')
    time=time.replace(':','_')
    time=time.replace('.','_')
    return str(date),str(time)

In [36]:
def create_database(crime_type):
    # Specify the current file path and the new file name
    current_file_path = "D:/hackathons/VigilAI/healthcare/healthcare/Eyebase/output.webm"
    city,coordinates=get_location()

    date,time=get_date_and_time()

    new_file_name =str(city)+'_'+date+'_'+time+'.webm'
    new_data = [new_file_name,crime_type,city,coordinates,date,time]

    csv_file = "D:/hackathons/VigilAI/healthcare/healthcare/AIbase.csv"

    with open(csv_file, 'a', newline='') as file:
        writer = csv.writer(file)

        # Append the new data to the CSV file
        writer.writerow(new_data)


    # Extract the directory and the extension from the current file path
    directory = os.path.dirname(current_file_path)
    extension = os.path.splitext(current_file_path)[1]

    # Create the new file path by combining the directory, new file name, and extension
    print(os.path.join(directory, new_file_name))
    new_file_path = os.path.join(directory, new_file_name)

    # Rename the file
    os.rename(current_file_path, new_file_path)
    # print("Threading time : ",end_time-start_time)

In [37]:
def add_to_database(crime_type):
    # Specify the source file path and the destination folder path
    source_file = "D:/hackathons/output.webm"
    destination_folder = "D:/hackathons/VigilAI/healthcare/healthcare/Eyebase"

    # Copy the file to the destination folder
    shutil.copy(source_file, destination_folder)
    create_database(crime_type)

In [38]:
import queue


In [39]:

# def predict_on_live_video(video_file_path, output_file_path, window_size):
#     input_buffer = queue.Queue()
#     frame_count=0
#     frames=[]
#     crime_type='NA'
#     counter=0
    
#     # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
#     predicted_labels_probabilities_deque = deque(maxlen = window_size)
 
#     # Reading the Video File using the VideoCapture Object
#     video_reader = cv2.VideoCapture(video_file_path)
    
    
    
#     # Getting the width and height of the video 
#     original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
#     original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
#     # Writing the Overlayed Video Files Using the VideoWriter Object
#     # video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc(*'VP80'), 24, (original_video_width, original_video_height))
 
#     while True: 
 
#         # Reading The Frame
#         status, frame = video_reader.read() 
 
#         if not status:
#             break
 
#         # Resize the Frame to fixed Dimensions
#         resized_frame = cv2.resize(frame, (image_height, image_width))
         
#         # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
#         normalized_frame = resized_frame / 255
 
#         # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
#         predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]
 
#         # Appending predicted label probabilities to the deque object
#         predicted_labels_probabilities_deque.append(predicted_labels_probabilities)
 
#         # Assuring that the Deque is completely filled before starting the averaging process
#         if len(predicted_labels_probabilities_deque) == window_size:
 
#             # Converting Predicted Labels Probabilities Deque into Numpy array
#             predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)
 
#             # Calculating Average of Predicted Labels Probabilities Column Wise 
#             predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
 
#             # Converting the predicted probabilities into labels by returning the index of the maximum value.
#             predicted_max=np.max(predicted_labels_probabilities_averaged)

#             predicted_label = np.argmax(predicted_labels_probabilities_averaged)

        #     if predicted_max>0.6 and predicted_label<10:
        #         predicted_label = np.argmax(predicted_labels_probabilities_averaged)
        #         frame_count=frame_count+1
        #         if frame_count<450:
        #             if counter==0:
        #                 crime_type=classes_list[predicted_label]
        #                 counter=1
        #             frames.append(frame)
        #             save_clip(frames)
        #         else:
        #             if len(frames)>100:    
        #                 frame_count=0
        #                 frames=[]
        #     else:
        #         predicted_label=10 

                
        #     # Accessing The Class Name using predicted label.
        #     predicted_class_name = classes_list[predicted_label]

           
        # # Writing The Frame
        # # video_writer.write(frame)
 
 
        # cv2.imshow('CCTV FOOTAGE', frame)
 
        # key_pressed = cv2.waitKey(1)
 
        # if key_pressed == ord('q'):
        #     break
            
#     add_to_database(crime_type)
#     cv2.destroyAllWindows()
     
#     # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
#     video_reader.release()
#     # video_writer.release()
    

In [40]:
# count=0
import threading
class test:
    def __init__ (self,video_file_path):
        self.frame_count = 0 
        self.frames = []
        self.status=0
        self.crime_type = "NA"
        self.counter = 0
        self.video_file_path=video_file_path
        self.predicted_labels_probabilities_deque = deque(maxlen = 30)
        self.video_reader = cv2.VideoCapture(self.video_file_path)
        self.processing_thread = None
        self.is_processing = False
        self.window_size=30
        self.count=0
        self.start_time = time.time()
        self.end_time = 0


    def start_processing(self):
        print(self.video_file_path)
        self.is_processing = True
        print("gbg")
        # video_reader = cv2.VideoCapture(self.video_file_path)
        self.processing_thread = threading.Thread(target=self.process_video)
        self.processing_thread.start()
        print(self.video_file_path)
        # print(video_reader)
        # self.status,self.frames = video_reader.read()
        print(self.status)
        # return frames

    def stop_processing(self):
        self.is_processing = False
        if self.processing_thread is not None:
            # self.processing_thread.join()
            pass
    
    def process_video(self):
        
        # original_video_width = int(self.frames(cv2.CAP_PROP_FRAME_WIDTH))
        # original_video_height = int(self.frames(cv2.CAP_PROP_FRAME_HEIGHT))
        print("kkk")
        while self.is_processing:
            print("lll")
            frames=self.frames
            status, frame = self.video_reader.read()
            if not status:
                print(self.status)
                break
            print("mmm")
            resized_frame = cv2.resize(frame, (image_height, image_width))
            normalized_frame = resized_frame / 255
            ti=threading.Thread(target=test.predict_on_live_video,args= [self,normalized_frame,frame])
            ti.start()
            # ti.join()
            # predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]
            # self.predicted_labels_probabilities_deque.append(predicted_labels_probabilities)
            # if len(self.predicted_labels_probabilities_deque) == self.window_size:
            #     predicted_labels_probabilities_np = np.array(self.predicted_labels_probabilities_deque)
            #     predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
            #     predicted_max=np.max(predicted_labels_probabilities_averaged)
            #     predicted_label = np.argmax(predicted_labels_probabilities_averaged)
            #     if predicted_max>0.6 and predicted_label<10:
            #         predicted_label = np.argmax(predicted_labels_probabilities_averaged)
            #         self.frame_count = self.frame_count + 1
            #         if frame_count<450:
            #             if counter == 0:
            #                 crime_type = classes_list[predicted_label]
            #                 counter = 1
            #             frames.append(frame)
            #             save_clip(frames)
            #         else:
            #             if len(frames)>100:
            #                 frame_count = 0
            #                 frames = []
            #     else:
            #         predicted_label= 10
            #     predicted_class_name = classes_list[predicted_label]
            cv2.imshow('CCTV FOOTAGE',frame)
            key_pressed = cv2.waitKey(1)
            if key_pressed == ord('q'):
                break
            add_to_database(self.crime_type)
            cv2.destroyAllWindows()
            print(threading.active_count())
            # video_reader.release()
        # t1 = threading.Thread(target=read_video,args=[])
        # t2 = threading.Thread(target=process_video,args=[])
        # t1.start()
        # t2.start()
        # t1.join()
        # t2.join()
    def predict_on_live_video(self,normalized_frame,frame,window_size=30):
    
        # predicted_labels_probabilities_deque = deque(maxlen = window_size
        print("predict",threading.active_count())
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]
        self.predicted_labels_probabilities_deque.append(predicted_labels_probabilities)
        if len(self.predicted_labels_probabilities_deque) == self.window_size:
                predicted_labels_probabilities_np = np.array(self.predicted_labels_probabilities_deque)
                predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
                predicted_max=np.max(predicted_labels_probabilities_averaged)
                predicted_label = np.argmax(predicted_labels_probabilities_averaged)
                if predicted_max>0.6 and predicted_label<10:
                    predicted_label = np.argmax(predicted_labels_probabilities_averaged)
                    self.frame_count = self.frame_count + 1
                    if frame_count<450:
                        if counter == 0:
                            crime_type = classes_list[predicted_label]
                            counter = 1
                        frames.append(frame)
                        save_clip(frames)
                    else:
                        if len(frames)>100:
                            frame_count = 0
                            frames = []
                else:
                    predicted_label= 10
                predicted_class_name = classes_list[predicted_label]
        # self.count=1
        # t2 = threading.Thread(target=test.process_video,args=[self])
        # t1 = threading.Thread(target=test.read_video,args=[self])
        # print(self.count)
        # t2.start()
        # t1.start()
        # # t1.join()
        # t2.join()
        




In [41]:
import tkinter as tk
from tkinter.filedialog import askopenfilename
root = tk.Tk()
# output_file_path = "C:/Users/rudra/OneDrive/Desktop"
# window_size = 30
# video_file_path = 0
def start():
    print("tkinter",threading.active_count())
    file_path = askopenfilename()
    output_file_path = "C:/Users/dwive/Desktop"
    window_size = 30
    print(file_path)
    video_file_path = file_path
    my_object = test(video_file_path)
    print("start")
    my_object.start_processing()
# predict_on_live_video(video_file_path, output_file_path, window_size)
# start()

In [42]:
# import tkinter as tk
# from tkinter.filedialog import askopenfilename
def button_click():
    file_path = askopenfilename()

    # Display the selected file path
    # print("Selected file path:", file_path)
    start(file_path)

# Create the Tkinter root window
button = tk.Button(root, text="Click Me!",command = start)
button.pack()
root.mainloop()

tkinter 6


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\rudra\AppData\Local\Temp\ipykernel_38948\3724728784.py", line 85, in process_video
  File "C:\Users\rudra\AppData\Local\Temp\ipykernel_38948\4076860344.py", line 7, in add_to_database
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\shutil.py", line 427, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundEr

C:/Users/rudra/Downloads/VigilAI/test_data/Fighting018_x264.mp4
start
C:/Users/rudra/Downloads/VigilAI/test_data/Fighting018_x264.mp4
gbg
kkk
lll
C:/Users/rudra/Downloads/VigilAI/test_data/Fighting018_x264.mp4
0
mmm
predict 8
1/1 [==============================] - 0s 112ms/step
